In [15]:
!pip install tensorflow_text
!pip install tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import pandas as pd

In [26]:
TRAINING_FILE='/content/drive/MyDrive//Topics/train_FDtest4.csv'
df=pd.read_csv(TRAINING_FILE)
# df['text'] = df['text'].apply(clean_data().clean)
x_train=df["text"]
y_train=df["label"]

#x_train , x_test , y_train , y_test =train_test_split(df['text'],df['label'],test_size=0.2, random_state=42, shuffle=True)


In [27]:
from sklearn.preprocessing import LabelBinarizer

label_as_binary = LabelBinarizer()
train__y_labels = label_as_binary.fit_transform(y_train)

In [22]:
train__y_labels[0]


array([0, 0, 1, 0, 0, 0, 0])

In [23]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [28]:
#from sklearn.model_selection import train_test_split
#import config
#from clean_data import clean_data
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

#test__y_labels = label_as_binary.fit_transform(y_test)

bert_preprocess = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang_preprocess/1")
bert_encoder = hub.KerasLayer("https://tfhub.dev/jeongukjae/smaller_LaBSE_15lang/1")

# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
layer1 = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
layer2 = tf.keras.layers.Dense(7, activation='softmax', name="output")(layer1)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [layer2])
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

METRICS = [
  tf.keras.metrics.BinaryAccuracy(name='accuracy'),
  tf.keras.metrics.Precision(name='precision'),
  tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer='adam',
          loss='categorical_crossentropy',
          metrics=METRICS)
EPOCHS=30
#for i in range(EPOCHS):
model.fit(x_train, train__y_labels, epochs=EPOCHS, callbacks=[callback])
#PATH_TO_SAVE = f"/content/drive/MyDrive/Topics/topics_classification/Epoch_{i}"
PATH_TO_SAVE = "/content/drive/MyDrive/Topics/topics_classification/Epoch_test4"
#model.evaluate(x_test, test__y_labels)
model.save(PATH_TO_SAVE)
#model = tf.keras.models.load_model(PATH_TO_SAVE)

Epoch 1/30
408/408 [==============================] - 139s 316ms/step - loss: 0.7179 - accuracy: 0.9373 - precision: 0.9064 - recall: 0.6259
Epoch 2/30
408/408 [==============================] - 130s 318ms/step - loss: 0.4383 - accuracy: 0.9596 - precision: 0.8985 - recall: 0.8084
Epoch 3/30
408/408 [==============================] - 130s 319ms/step - loss: 0.4020 - accuracy: 0.9626 - precision: 0.9007 - recall: 0.8298
Epoch 4/30
408/408 [==============================] - 130s 318ms/step - loss: 0.3827 - accuracy: 0.9644 - precision: 0.9039 - recall: 0.8399
Epoch 5/30
408/408 [==============================] - 130s 319ms/step - loss: 0.3774 - accuracy: 0.9635 - precision: 0.9000 - recall: 0.8378
Epoch 6/30
408/408 [==============================] - 130s 318ms/step - loss: 0.3680 - accuracy: 0.9651 - precision: 0.9017 - recall: 0.8483
Epoch 7/30
408/408 [==============================] - 130s 318ms/step - loss: 0.3611 - accuracy: 0.9659 - precision: 0.9035 - recall: 0.8520
Epoch 8/30
40

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Topics/topics_classification/Epoch_test4/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Topics/topics_classification/Epoch_test4/assets


In [30]:
SAVE_PATH="/content/drive/MyDrive/Topics/topics_classification/Epoch_test4"
TEST_FILE='/content/drive/MyDrive//Topics/test_FDtest4.csv'

In [31]:
from sklearn.preprocessing import LabelBinarizer
#import sys
label_as_binary = LabelBinarizer()
model = tf.keras.models.load_model(SAVE_PATH)
test=pd.read_csv(TEST_FILE)
x_test=test['text']
y_test=test['label']
test_y_labels = label_as_binary.fit_transform(y_test)

model.evaluate(x_test, test_y_labels)

102/102 [==============================] - 33s 312ms/step - loss: 0.3894 - accuracy: 0.9634 - precision: 0.8905 - recall: 0.8477


[0.38940298557281494,
 0.9633553624153137,
 0.8905344605445862,
 0.8476861715316772]

In [46]:
#model = tf.keras.models.load_model(SAVE_PATH)
def run(model, comment):
    resultat=model.predict([comment])
    resultat=resultat.tolist()[0]
    #if max(resultat) > 0.6:
    i= resultat.index(max(resultat))
    #else:
     # i= 6
    return i+1

TOPICS  = {1:'economie',2:'environnement',3:'immigration_labeled',
           4:'laicite',5:'sante',6:'sécurité',7:'autre'}

In [34]:
run(model,'encore faut il avoir un depute rn dans nos circonscriptions')

1/1 [==============================] - 0s 40ms/step


7

In [39]:


classes=pd.read_csv("/content/drive/MyDrive/Topics/test8class_kw.csv")

In [40]:
classes.head()

,text,updated_label,id_class
0,Organisation mondiale de la Santé (OMS) est l'...,santé,5.0
1,L’importance de la lutte contre les maladies t...,santé,5.0
2,utte contre les maladies transmissibles est au...,santé,5.0
3,il est courant d’affirmer que la charge de mal...,santé,5.0
4,si les campagnes des autorités sanitaires ne s...,santé,5.0


In [36]:
di={1:'economie',2:"immigration",3:"sec",4:'santé',5:"environnement",6:"retraite",7:"inégalité",8:"laicité"}

In [37]:
classes['updated_label']=classes['label'].map(di)

NameError: ignored

In [ ]:
classes=classes[["text", "updated_label"]]

In [47]:
df=pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/data_filter2.csv")

In [ ]:
df.head()

,date_extraction,date_publication,comment,id_source,id_groupe,score,avis
0,2022-05-20,2022-05-16 00:00:00,mr macron à trouvé sont bras droit,1,3,0.810707,1
1,2022-05-20,2022-05-20 00:00:00,il a choisi de mettre lepen comme première opp...,1,6,0.616435,1
2,2022-05-20,2022-05-16 00:00:00,en fait c'est macron lui-même le vrai premier ...,1,3,0.614738,1
3,2022-05-20,2022-05-13 12:23:41,pauvre macron il est à la recherche d'un mini...,1,3,0.686973,1
4,2022-05-20,2022-05-06 12:23:52,pour favoriser une cadre de la fi,1,2,0.774590,1


In [ ]:
%%time
for i in range(len(df)):
  res=run(model,df['comment'].iloc[i])
  # a= res[0].tolist()
  # im=a.index(max(a))
  # ec1["pred"].iloc[i]=im
  # ec1['list'].iloc[i]=a
#  ec1.insert(2, "pred", im)
 # ec1.insert(3, "list", a)
  df.loc[i, "id_class"]=res

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - ETA: 0s

In [ ]:
df.to_csv("/content/drive/MyDrive/Topics/res_test4.csv",index=False)

In [42]:
classes.to_csv("/content/drive/MyDrive/Topics/res8class_threshold_test4.csv",index=False)